In [74]:
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image
from colour.models import eotf_ST2084

files = [
    (None, '/home/matt/dev/github/3ll3d00d/jrmc-utils/greyscale/greyscale_100_1000.png'),
    (2.2, '/media/home-media/docs/calibration/tonemapping/greyscale/greyscale_jrvr_spline0.5_peak1000_3dlut_none.png'),
    (2.2, '/home/matt/Pictures/orient_express.png'),
    (2.2, '/home/matt/Pictures/orient_express_full.png'),
    (2.2, '/home/matt/Pictures/showhorses.png'),
    # (2.2, '/home/matt/Downloads/adl5.png')
]
data = []
for gamma, f in files:
    im = np.array(Image.open(f).convert('RGB'))
    rgb = im / 255
    linear_rgb = rgb ** gamma if gamma else rgb
    luminance = np.dot(linear_rgb[..., :3], [0.2126, 0.7152, 0.0722])
    nits_luminance = eotf_ST2084(luminance)

    apl = np.mean(luminance) * 100
    fall = np.mean(nits_luminance)
    pq_fall = eotf_ST2084(np.mean(luminance))
    p9995 = np.percentile(nits_luminance, 99.95)
    pq_p9995 = eotf_ST2084(np.percentile(luminance, 99.95))
    max_nits = np.max(nits_luminance)
    max_xy = list(reversed(np.unravel_index(luminance.argmax(), luminance.shape)))
    data.append([Path(f).name, np.max(im), apl, fall, pq_fall, p9995, pq_p9995, max_nits, max_xy])

df = pd.DataFrame(data, columns=['Name', 'max(RGB)', 'APL %', 'FALL', 'PQ FALL', 'P99.95', 'PQ P99.95', 'MAX', 'MAX XY'])
df


,Name,max(RGB),APL %,FALL,PQ FALL,P99.95,PQ P99.95,MAX,MAX XY
0,greyscale_100_1000.png,192,37.558007,145.211965,24.728669,1010.272079,1010.272079,1010.272079,"[3800, 0]"
1,greyscale_jrvr_spline0.5_peak1000_3dlut_none.png,255,25.532285,467.134678,5.552513,10000.000000,10000.000000,10000.000000,"[3800, 0]"
2,orient_express.png,255,10.777621,18.345741,0.395969,1959.765317,1959.765317,8911.810420,"[1783, 107]"
3,orient_express_full.png,255,7.794570,13.255757,0.171804,1704.627607,1704.627318,9210.303265,"[1864, 240]"
4,showhorses.png,255,60.525645,1133.125275,256.471106,7611.028205,7611.028205,10000.000000,"[1009, 1033]"
